In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import numpy as np

In [5]:
# Define reference state in J, mol, K, Pa
P_ref = 0.01 * 10**6
T_ref = 45 + 273
V_ref = 14.67 * 0.018
U_ref = 2437.2 * 18
H_ref = 2583.9 * 18
S_ref = 8.149 * 18

# Parameters for EOS
Cp = 1.996 * 18
Tc = 647.3
Pc = 22.09 * 10**6
a = 27/64 * 8.314**2 * Tc**2 / Pc
b = 1/8 * 8.314 * Tc / Pc

def predict(P, T):
  # Define starting quantities
  P_cal = P_ref
  T_cal = T_ref
  V_cal = V_ref
  U_cal = U_ref
  H_cal = H_ref
  S_cal = S_ref

  # Convert P to Pascal, and T to Kelvin
  P = P * 10**6
  T = T + 273

  # Integrate with respect to pressure using rectangular method
  points = np.linspace(P_cal, P, 10**5, False)
  dP = points[1] - points[0]
  for i in range(len(points)):
    P = points[i]
    dVdP = V_cal**2 * (b - V_cal) / (3*P*V_cal**2 + a - 2*b*P*V_cal - 2*8.314*T_cal*V_cal)
    dVdT = 8.314*V_cal**2 / (3*P*V_cal**2 + a - 2*b*P*V_cal - 2*8.314*T_cal*V_cal)
    V_next = V_cal + dVdP * dP
    H_next = H_cal + (-T_cal * dVdT + V_cal) * dP
    S_next = S_cal + (-dVdT) * dP

    V_cal = V_next
    H_cal = H_next
    S_cal = S_next

  P_cal = points[-1] + dP
  U_cal = H_cal - (P_cal * V_cal)

  # Integrate with respect to temperature using rectangular method
  points = np.linspace(T_cal, T, 10**5, False)
  dT = points[1] - points[0]
  for i in range(len(points)):
    T = points[i]
    dVdT = 8.314*V_cal**2 / (3*P_cal*V_cal**2 + a - 2*b*P_cal*V_cal - 2*8.314*T*V_cal)
    V_next = V_cal + dVdT * dT
    H_next = H_cal + Cp * dT
    S_next = S_cal + (Cp / T) * dT

    V_cal = V_next
    H_cal = H_next
    S_cal = S_next
    
  T_cal = points[-1] + dT
  U_cal = H_cal - (P_cal * V_cal)

  # Convert back to kJ, kg, C, MPa
  P_cal = P_cal / 10**6
  T_cal = T_cal - 273
  V_cal = V_cal / 0.018
  U_cal = U_cal / 18
  H_cal = H_cal / 18
  S_cal = S_cal / 18

  return np.array([P_cal, T_cal, V_cal, U_cal, H_cal, S_cal])

print(predict(0.03, 100))

[3.00000000e-02 1.00000000e+02 5.71652173e+00 2.52075088e+03
 2.69224653e+03 7.95713408e+00]
